In [3]:
import os, sys
sys.path.append(os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd()))))


In [6]:
import pickle
import numpy as np

In [4]:


# tensorflow 
from asynfed.client.frameworks.tensorflow.tensorflow_framework import TensorflowFramework
from data_preprocessing import TensorflowDataPreprocessing
from Lenet import LeNet


# Preprocessing data
# mnist dataset
# Set the file paths for the MNIST digit dataset files
train_images_path = './data/mnist_data/train-images-idx3-ubyte.gz'
train_labels_path = './data/mnist_data/train-labels-idx1-ubyte.gz'
test_images_path = './data/mnist_data/t10k-images-idx3-ubyte.gz'
test_labels_path = './data/mnist_data/t10k-labels-idx1-ubyte.gz'



# preprocessing data to be ready for low level tensorflow training process
data_preprocessing = TensorflowDataPreprocessing(train_images_path=train_images_path,
                                                 train_labels_path=train_labels_path, batch_size=64, split=True,
                                                 fract=0.2, evaluate_images_path=test_images_path,
                                                 evaluate_labels_path=test_labels_path)


# define dataset
train_ds = data_preprocessing.train_ds
test_ds = data_preprocessing.test_ds
evaluate_ds = data_preprocessing.evaluate_ds

data_size = 10000

# define model
lenet_model = LeNet(input_features = (32, 32, 1), output_features = 10)
# define framework
tensorflow_framework = TensorflowFramework(model = lenet_model, data_size= data_size, train_ds= train_ds, test_ds= test_ds)


2023-05-23 23:26:26.032514: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-23 23:26:26.095511: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-23 23:26:26.096924: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-23 23:26:27.022419: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-05-23 23:26:28.655285: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-23 23:26:28.655693: W tensorflow/core/common_r

In [5]:
tensorflow_framework

In [42]:
# load sample of previous, current and global weights
previous_weights_path = './testweight_v0.pkl'
current_weights_path = './testweight_v1.pkl'
global_weights_path = './testweight_v2.pkl'

with open(previous_weights_path, "rb") as f:
    previous_weights = pickle.load(f)

with open(current_weights_path, "rb") as f:
    current_weights = pickle.load(f)

with open(global_weights_path, "rb") as f:
    global_weights = pickle.load(f)



In [43]:
# check to see whether there is any conflict of the dim of these weights
i = 0
for (previous_layer, current_layer, global_layer) in zip(previous_weights, current_weights, global_weights):
    if previous_layer.shape == current_layer.shape == global_layer.shape:
        pass
    else:
        print(f"conflict in dim at layer {i}") 
        print(f"previous_layer shape, current_layer shape, global_layer shape: {previous_layer.shape} {current_layer.shape} {global_layer.shape}") 
    i+= 1

In [47]:
# calcuate direction for updating process
e_local = [layer_a - layer_b for layer_a, layer_b in zip(current_weights, previous_weights)]
print(e_local[1])
e_global = [layer_a - layer_b for layer_a, layer_b in zip(global_weights, current_weights)]
print(e_global[1])
direction = [np.multiply(a, b) for a, b in zip(e_local, e_global)]
print(direction[1])

[ 0.00366509  0.01329409  0.00529997 -0.00952246  0.00763258 -0.02961813]
[-0.00283897 -0.01276677  0.00068362  0.00858577 -0.01409635  0.02698018]
[-1.04050905e-05 -1.69722567e-04  3.62316882e-06 -8.17576874e-05
 -1.07591564e-04 -7.99102359e-04]


In [49]:
# initialize an empty merged result
merged_weights = [np.zeros(layer.shape) for layer in current_weights]
alpha = 0.4

In [50]:
i = 0
for (local_layer, global_layer, direction_layer) in zip(current_weights, global_weights, direction):
    merged_array = np.where(direction_layer >=0, global_layer, (1 - alpha) * global_layer + alpha * local_layer)
    merged_weights[i] = merged_array
    i+=1


In [55]:
print("previous weights:", previous_weights[1])
print("current weights:", current_weights[1])
print("global weights:", global_weights[1])
print("merged weights:", merged_weights[1])

previous weights: [ 0.04069237  0.02736644 -0.19731933 -0.09315923  0.16284755 -0.21118534]
current weights: [ 0.04435746  0.04066053 -0.19201936 -0.10268169  0.17048013 -0.24080347]
global weights: [ 0.04151849  0.02789376 -0.19133574 -0.09409592  0.15638378 -0.21382329]
merged weights: [ 0.04265407  0.03300047 -0.19133574 -0.09753023  0.16202232 -0.22461537]
